<center> <h1>PRIMERA PREDICIÓN DEL SUEÑO DE UN PACIENTE</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>

[URL PHYSIONET](https://physionet.org/content/sleep-accel/1.0.0/)

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statistics as stats

In [2]:
pacientes_sueño=[]
for file in (os.listdir("labels")):
    pacientes_sueño.append(os.path.join("labels",file))
    pacientes_sueño=sorted(pacientes_sueño)
    
pacientes_frec_card=[]
for file in os.listdir("heart_rate"):
    pacientes_frec_card.append(os.path.join("heart_rate",file))

pacientes_pasos=[]
for file in os.listdir("steps"):
    pacientes_pasos.append(os.path.join("steps",file))

pacientes_movimiento=[]
for file in os.listdir("motion"):
    pacientes_movimiento.append(os.path.join("motion",file))

La siguiente celda de código se corresponde con la ventana deslizante que recorrerá la señal fisiológica deseada. La función toma como datos de entrada el tamaño de la ventana en segundos y la señal sobre la que se quiere trabajar.

In [3]:
def get_window(w_size,signal,n_sueño):
    time = signal["Tiempo"]
    f_list=[]
    
    for i in range(n_sueño):
        a = w_size*(1+i)
        b = w_size*(2+i)

        idx=[]
        for t in time:
            if t>a and t<b:
                idx.append(True)
            else:
                idx.append(False)
        
        f=signal[idx]
        f_list.append(f)
        
    return f_list

En todos los casos, se seleccionará una w_size de 30 segundos para tener una característica cada 30s y así coincidir la cantidad de los datos adquiridos por medio del Apple Watch con los datos recogidos por la PSG.

Con la señal de frecuencia cardíaca se han decidido sacar 3 vectores de características: mínimo, máximo y media.

## FEATURES

* **feat1: min_list**. La primera característica que se extrae es la lista de frecuencias mínimas.
* **feat2: max_list**. La segunda característica que se extrae es la lista de frecuencias máximas.
* **feat3: mean_list**. La tercera característica que se extrae es la lista de frecuencias medias.

## OUTCOME

* **labels:**. Lista de etiquetas del sueño

# IDENTIFICACION
* **ID:**. Variable que identifica a cada paciente.

# MATRICES HIJAS

Realizamos esta operación sobre todos los pacientes

In [4]:
matrices_hija=[]
for a, (i,j) in  enumerate(zip(pacientes_sueño,pacientes_frec_card)):
    #SUEÑO
    sueño=pd.read_csv(i, delimiter=' ')
    sueño.columns=["Tiempo","Etiqueta"]

    #FRECUENCIA CARDIACA
    frec_card=pd.read_csv(j)
    frec_card.columns=["Tiempo","Frec card"]
    
    w_size=30
    signal=frec_card
    n_sueño=len(sueño)
    f_list= get_window(w_size,signal,n_sueño)

    mean_list=[]
    min_list=[]
    max_list=[]

    for f in f_list:
        mean_list.append(round(f['Frec card'].mean(),2))
        min_list.append(round(f['Frec card'].min(),2))
        max_list.append(round(f['Frec card'].max(),2))
    
    feat1=min_list
    feat2=max_list
    feat3=mean_list
    labels=sueño["Etiqueta"]
    id_list=[a]*len(labels)
    
    datos = {'Min_hr': feat1, "Max_hr": feat2,"Mean_hr":feat3,"Etiquetas":labels,"ID_P":id_list}
    datos_paciente = pd.DataFrame(data=datos)
    matrices_hija.append(datos_paciente)

In [24]:
matrices_hija[6]

,Min_hr,Max_hr,Mean_hr,Etiquetas,ID_P
0,82,84,83.00,-1,6
1,75,80,78.00,-1,6
2,75,80,76.50,-1,6
3,70,75,71.80,-1,6
4,68,71,69.67,-1,6
...,...,...,...,...,...
963,59,66,62.33,-1,6
964,56,59,57.50,-1,6
965,57,63,60.60,-1,6
966,61,64,62.50,-1,6


Una vez que ta tenemos guardadas todas las matrices, nos disponemos a crear un clasificador SVM. Primero vamos a utilizar 20 pacientes para entrenar el modelo; por ejemplo, seleccionaremos las 20 primeras matrices hijas; y 11 matrices de pacientes para testear el modelo.

Esto es una funcion para limpiar el dataframe de Nan y valores inf.

In [6]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [18]:
#TRAIN
Training=pd.concat(matrices_hija[0:21]) 
Training= clean_dataset(Training) # Tratar los Nan porque da errores en el calsificador
Train=Training.drop('Etiquetas',axis=1) 

X_train=Train.drop('ID_P',axis=1)
y_train=Training["Etiquetas"]

#TEST
Test=pd.concat(matrices_hija[22:])  
Test=clean_dataset(Test)            # Tratar los Nan porque da errores en el calsificador
Test_f=Test.drop('Etiquetas',axis=1)
X_test=Test_f.drop('ID_P',axis=1)
y_test=Test["Etiquetas"]


# Creación modelo y evaluación

In [19]:
#CREACION
svc_clf = SVC(C=10**5, kernel='rbf',gamma=0.0001)
svc_clf.fit(X_train, y_train)

SVC(C=100000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [20]:
predict = svc_clf.predict(X_test)
#EVALUACION

acc_test= svc_clf.score(X_test, y_test).round(4)
acc_train= svc_clf.score(X_train, y_train).round(4)
print('La exactitud para el modelo de SVM en el conjunto de TEST es:',acc_test,'\n')
print('La exactitud para el modelo de SVM en el conjunto de TRAIN es:',acc_train,'\n')

print('Confusion Matrix')
print(confusion_matrix(y_test, predict),'\n')
#print("=============== Classification Report ================")
#print(classification_report(y_test, predict))

La exactitud para el modelo de SVM en el conjunto de TEST es: 0.4706 

La exactitud para el modelo de SVM en el conjunto de TRAIN es: 0.5084 

Confusion Matrix
[[   0    2    0  137    2    0   13]
 [   0   38    1  570    8    0   82]
 [   0   51    2  538    3    0   79]
 [   0   15    0 3765    2    0  135]
 [   0    2    0  965    0    0   15]
 [   0    0    0  166    0    0    0]
 [   0   17    2 1646   13    0  163]] 



In [25]:
matriz_confusion=pd.crosstab(y_test,predict, rownames=['Clase predicha'], colnames=['Resultado del clasificador'])

In [26]:
matriz_confusion

Resultado del clasificador,0.0,1.0,2.0,3.0,5.0
Clase predicha,,,,,
-1.0,2,0,137,2,13
0.0,38,1,570,8,82
1.0,51,2,538,3,79
2.0,15,0,3765,2,135
3.0,2,0,965,0,15
4.0,0,0,166,0,0
5.0,17,2,1646,13,163
